In [36]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
# import Connection

class App:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver conn when you're done with it
        self.driver.close()

    def test_conn(self):
        with self.driver.session() as session:
            try:
                session.run("Match () Return 1 Limit 1")
                print('connection ok')
            except Exception:
                print('connection error')
                
    def create_ingredientCategory(self, category_name):
        with self.driver.session() as session:
            res = session.write_transaction(
                self._create_and_return_ingredientCategory, category_name)
            for row in res:
                print("Created ingredient category: {c1}".format(c1=row['c1']))
    
    @staticmethod        
    def _create_and_return_ingredientCategory(tx, cn):
        query = (
            "MERGE (c1:IngredientCategory { name: $cn }) "
            "RETURN c1"
        )
        result = tx.run(query, cn=cn)
        try:
            return [{"c1": row["c1"]["name"]}
                    for row in result]
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise
        
    def create_ingredient_ingredientCategory_relationship(self, ingredient, ingredientCategory):
        with self.driver.session() as session:
            res = session.write_transaction(
                self._create_and_return_ingredient_ingredientCategory_relationship, ingredient, ingredientCategory)
            for row in res:
                print("Created 'IS_A' relationship between: {i}, {ic}".format(i=row['i'], ic=row['ic']))
                
    # @staticmethod
    # def _create_and_return_ingredient_ingredientCategory_relationship(tx, ingredient, ingredientCategory):
    #     query = (
    #         "MERGE (i:Ingredient { name: $ingredient }) "
    #         "MERGE (ic:IngredientCategory { name"
    #     )

## Testing

In [38]:
if __name__ == '__main__':
    #Aura queries use an encrypted connection using the "neo4j+s" URI scheme
    uri = "neo4j+s://34781963.databases.neo4j.io"
    user = "neo4j"
    password = "G556E8m9-XS5mCOSJScbv7dBja-vuLPnXgBIYiVGM1U"
    app = App(uri, user, password)
    # Functions to test go below here
    
    app.test_conn()

    app.create_ingredientCategory(category_name="grain")

    app.close()

connection ok
Created ingredient category: grain
